In [3]:
import json
from flask import Flask, request, jsonify
import pickle
import pandas as pd
import os
import sqlite3

In [26]:
data = pd.read_csv('data/Advertising.csv', index_col=False)
data

,Unnamed: 0,TV,radio,newpaper,sales
0,0,230.1,37.8,6s9.2,22100.0
1,1,44.5,39.3,45.1,10400.0
2,2,17.2,45.9,69.3,9300.0
3,3,151.5,41.3,58.5,18500.0
4,4,180.8,10.8,58.4,12900.0
...,...,...,...,...,...
195,195,38.2,3.7,13.8,7600.0
196,196,94.2,4.9,8.1,9700.0
197,197,177.0,9.3,6.4,12800.0
198,198,283.6,42.0,66.2,25500.0


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  200 non-null    int64  
 1   TV          200 non-null    float64
 2   radio       200 non-null    float64
 3   newpaper    200 non-null    object 
 4   sales       200 non-null    float64
dtypes: float64(3), int64(1), object(1)
memory usage: 7.9+ KB


In [28]:
index = data[data["newpaper"].str.contains(r".*[a-zA-Z].*")].index

In [29]:
data.loc[index,"newpaper"] = data.loc[index,"newpaper"].str.replace("s","")

In [30]:
data['newpaper'] = data['newpaper'].astype(float)

In [31]:
data.drop(columns='Unnamed: 0', inplace=True)


In [32]:
data.rename(columns={"newpaper" : "newspaper"}, inplace=True)

In [33]:
model = pickle.load(open('data/advertising_model','rb'))

X_test = data[["TV", "radio", "newspaper"]]

prediction = model.predict(X_test)


c:\Users\javie\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator Ridge from version 1.0.2 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [37]:
conn = sqlite3.connect('advertising_new.db')
cursor = conn.cursor()

In [38]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tablas = cursor.fetchall()
for t in tablas:
    print(t)

('advertising',)
('sqlite_sequence',)


In [39]:
cursor.execute("SELECT * FROM advertising WHERE type='table'")

OperationalError: no such column: type

In [40]:
columns = [descripcion[0] for descripcion in cursor.description]
columns

TypeError: 'NoneType' object is not iterable

In [41]:
query =\
"""
    SELECT tv, radio, newspaper FROM advertising
"""
data = cursor.execute(query)
results = data.fetchall()
results

[]

In [42]:
query =\
"""
    SELECT TV, radio, newspaper FROM advertising
"""
data = cursor.execute(query)
for a in data:
    print(a)

In [22]:

columns = [descripcion[0] for descripcion in cursor.description]
X_test = pd.DataFrame(results, columns=columns)

X_test

,TV,radio,newspaper


In [66]:
prediction = model.predict(X_test)

,tv,radio,newspaper
0,230.1,37.8,69.2
1,44.5,39.3,45.1
2,17.2,45.9,69.3
3,151.5,41.3,58.5
4,180.8,10.8,58.4
...,...,...,...
195,38.2,3.7,13.8
196,94.2,4.9,8.1
197,177.0,9.3,6.4
198,283.6,42.0,66.2


In [67]:
prediction = model.predict(X_test)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- tv
Feature names seen at fit time, yet now missing:
- TV


In [36]:
conn.close()

In [34]:
data.to_sql(name="advertising", con=conn, if_exists="append", index=False)


200